# Aggregated Binomial

In [1]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt

from scipy import stats
from scipy.special import expit as logistic
from scipy.special import softmax

%config InlineBackend.figure_format = 'retina'
warnings.simplefilter(action="ignore", category=FutureWarning)
RANDOM_SEED = 8927
np.random.seed(286)

In [2]:
az.style.use("arviz-darkgrid")
az.rcParams["stats.credible_interval"] = 0.89


def standardize(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()

In the chimpanzees data context, the models all calculated the liklelihood of observing either zero or one pulls of the left hand level. The models did so because the data was organised such that each row describes the outcome of a single pull. But in principle the same data could be organised differently. As long as we don't care about the order of the individual pulls, the same information is contained in a count of how many times each individual pulled the left hand level, for each combination of predictor variables. 


In [9]:
d = pd.read_csv("chimpanzees.csv", sep=";")
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition

d_aggregated = (
    d.groupby(["treatment", "actor"]).sum().reset_index()[["treatment", "actor", "pulled_left"]]
)

actor_idx, actors = pd.factorize(d.actor)
treat_idx, treatments = pd.factorize(d.treatment)

d_aggregated.head(10)




,treatment,actor,pulled_left
0,0,0,6
1,0,1,18
2,0,2,5
3,0,3,6
4,0,4,6
5,0,5,14
6,0,6,14
7,1,0,9
8,1,1,18
9,1,2,11


the pulled_left column on the right is the count of times each actor pulled the left hand level for trials in each treatment. Now we can get the same inferences as before, just by defining the following model.

In [10]:
with pm.Model() as m11_6:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    p = pm.Deterministic("p", pm.math.invlogit(a[d_aggregated.actor] + b[d_aggregated.treatment]))

    pulled_left = pm.Binomial("pulled_left", 18, p, observed=d_aggregated.pulled_left)

    trace_11_6 = pm.sample(random_seed=RANDOM_SEED)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [00:03<00:00, 1154.36draws/s]


Whats the bottom line? If you want to calculate WAIC and PSIS, you should use a logistic regression data format, not an aggregated format. Otherwise you are implicitly assuming that only large chunks of the data are seperable. There are time when this makes sense, like with multilevel models. But it doesnt in most ordinary binomial regressions.

# Aggregated Binomial - Graduate School Admissions


In the aggregated example above, the number of trials was always 18 in every row. This is often now the case. The way to handle this is to insert a variable from the data in place of the "18". Lets work through an example.

In [11]:
d_ad = pd.read_csv("UCBadmit.csv", sep=";")
d_ad

,dept,applicant.gender,admit,reject,applications
1,A,male,512,313,825
2,A,female,89,19,108
3,B,male,353,207,560
4,B,female,17,8,25
5,C,male,120,205,325
6,C,female,202,391,593
7,D,male,138,279,417
8,D,female,131,244,375
9,E,male,53,138,191
10,E,female,94,299,393


Our job is to evaluate whether these data contain evidence of gender bias in admissions. We will model the admissions decisions, focusing on applicant gender as a predictor variable. So we want to fit a binomial regression that models ADMIT as a function of each applicants gender. This will estimate the association between gender and probability of admission. This is what the model will look like, in mathematical form.

a = Binomial(N,p)
logit(p) = alpha(gid)
alpha(gid) = Normal(0,1.5)

The index variable GID indexes the gender of an applicant

In [14]:
gid = (d_ad["applicant.gender"] == "female").astype(int).values

with pm.Model() as m11_7:
    a = pm.Normal("a", 0, 1.5, shape=2)
    p = pm.Deterministic("p", pm.math.invlogit(a[gid]))

    admit = pm.Binomial("admit", p=p, n=d_ad.applications, observed=d_ad.admit)

    trace_11_7 = pm.sample(random_seed=RANDOM_SEED)
az.summary(trace_11_7, var_names=["a"], round_to=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [00:03<00:00, 1260.21draws/s]


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a[0],-0.22,0.04,-0.30,-0.15,0.0,0.0,1588.76,1546.88,1567.81,1004.35,1.0
a[1],-0.83,0.05,-0.92,-0.74,0.0,0.0,1986.45,1972.17,1986.72,1506.73,1.0


The posterior for male applicants, A[0] ios higher than that of female applicants. How much higher? We need to compute the contrast. Lets calculate the contrast on the logit scale as well as the contrast on the outcome scale

In [16]:
diff_a = trace_11_7["a"][:, 0] - trace_11_7["a"][:, 1]
diff_p = logistic(trace_11_7["a"][:, 0]) - logistic(trace_11_7["a"][:, 1])
az.summary({"diff_a - Relative": diff_a, "diff_p - Absolute": diff_p}, kind="stats", round_to=2)

,mean,sd,hpd_3%,hpd_97%
diff_a - Relative,0.61,0.06,0.49,0.73
diff_p - Absolute,0.14,0.01,0.11,0.17


The log-odds difference is certainly positive, corresponding to a higher probability of admission for male applicants. On the probability scale itself, the difference is somewhere between 12% and 16%.

Before moving on to speculate the cause of the male advantage, lets plot posterior predictions for the model

with m11_7:
    ppc = pm.sample_posterior_predictive(trace_11_7, random_seed=RANDOM_SEED, var_names=["admit"])[
        "admit"
    ]
pp_admit = ppc / d_ad.applications.values[None, :]

In [22]:
d_ad

,dept,applicant.gender,admit,reject,applications
1,A,male,512,313,825
2,A,female,89,19,108
3,B,male,353,207,560
4,B,female,17,8,25
5,C,male,120,205,325
6,C,female,202,391,593
7,D,male,138,279,417
8,D,female,131,244,375
9,E,male,53,138,191
10,E,female,94,299,393



# First plot the real data
for i in range(6):
    x = 1 + 2 * i

    y1 = d_ad.admit[x] / d_ad.applications[x]
    y2 = d_ad.admit[x + 1] / d_ad.applications[x + 1]

    plt.plot([x, x + 1], [y1, y2], "-C0o", alpha=0.6, lw=2)
    plt.text(x + 0.25, (y1 + y2) / 2 + 0.05, d_ad.dept[x])

    
# Then plot the predictions
plt.plot(range(1, 13), trace_11_7["p"].mean(0), "ko", fillstyle="none", ms=6, alpha=0.6)
plt.plot([range(1, 13), range(1, 13)], az.hpd(trace_11_7["p"]).T, "k-", lw=1, alpha=0.6)
plt.plot([range(1, 13), range(1, 13)], az.hpd(pp_admit).T, "k+", ms=6, alpha=0.6)

plt.xlabel("case")
plt.ylabel("admit")
plt.title("Posterior validation check")
plt.ylim(0, 1);


As shown above, these are pretty terrible predictions. There are only two departments in which females had a lower rate of admission than males (C and E) and yet the model says that femles should expect to have a 14% lower chance of admission.

The model did correctly answer the question: 'what are the probabilities of admission for females and males, across all departments?' The problem in this case is that males and females do not apply for the same departments, and departments vary in their rate of admission. This makes tha answer misleading. You can see the steady decline in admisson probability for both males and females from department A to department F. Females in these data tended not to apply to departments like A and B, instead they applies in large numbers to departments like F.

So while it is true overall that females had a lowere probability of admission in this data, it is clearly not true within most deparments. And not that just inspecting the posterior distribution alone would not have revealed this to us. We had to do a posterior validation check.

Instead of asking "What are the average probabilities of admission for females and males across all departments?" we want to ask "what is the average probability of admission between females and males within departments? In order to ask the second question we estimate unique female and male admission rates in each department. 

In [33]:
dept_id = pd.Categorical(d_ad["dept"]).codes

with pm.Model() as m11_8:
    a = pm.Normal("a", 0, 1.5, shape=2)
    delta = pm.Normal("delta", 0, 1.5, shape=6)

    p = pm.math.invlogit(a[gid] + delta[dept_id])

    admit = pm.Binomial("admit", p=p, n=d_ad.applications, observed=d_ad.admit)

    trace_11_8 = pm.sample(2000, random_seed=RANDOM_SEED)
az.summary(trace_11_8, round_to=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [delta, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 10000/10000 [00:12<00:00, 824.25draws/s]
The number of effective samples is smaller than 10% for some parameters.


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a[0],-0.55,0.54,-1.49,0.52,0.02,0.02,792.76,606.06,801.01,736.10,1.01
a[1],-0.45,0.54,-1.43,0.58,0.02,0.02,788.91,566.63,796.86,724.65,1.01
delta[0],1.13,0.54,0.04,2.07,0.02,0.01,797.49,724.15,807.34,746.49,1.01
delta[1],1.08,0.54,0.05,2.06,0.02,0.01,795.57,713.65,804.60,755.14,1.01
delta[2],-0.13,0.54,-1.18,0.83,0.02,0.02,784.67,575.85,792.62,729.20,1.01
delta[3],-0.16,0.54,-1.19,0.82,0.02,0.02,794.57,579.11,804.05,756.48,1.01
delta[4],-0.61,0.55,-1.65,0.38,0.02,0.01,804.35,798.79,811.34,737.27,1.01
delta[5],-2.16,0.55,-3.16,-1.12,0.02,0.01,830.72,830.72,836.55,738.53,1.01


In [35]:
diff_a = trace_11_8["a"][:, 0] - trace_11_8["a"][:, 1]
diff_p = logistic(trace_11_8["a"][:, 0]) - logistic(trace_11_8["a"][:, 1])
az.summary({"diff_a - Relative": diff_a, "diff_p - Absolute": diff_p}, kind="stats", round_to=2)

,mean,sd,hpd_3%,hpd_97%
diff_a - Relative,-0.10,0.08,-0.24,0.07
diff_p - Absolute,-0.02,0.02,-0.06,0.01
